<a href="https://colab.research.google.com/github/SBiscardi/cd-202202-santiago-biscardi/blob/master/Taller7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyproj
!pip install geopandas
!sudo apt install libspatialindex-dev
!pip install rtree

!pip install cython
!pip install cartopy

!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall -y shapely    # cartopy and shapely aren't friends (early 2020)
!pip install shapely --no-binary shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 13.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 12.5 MB/s 
     |████████████████████████████████| 16.7 MB 70.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ub

In [2]:
import geopandas
import pandas as pd
import numpy as np
import geopandas as gpd
import urllib
from decimal import DivisionByZero
from math import sqrt

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import cartopy
import cartopy.crs as ccrs                   # for projections
import cartopy.feature as cfeature           # for features
import cartopy.io.shapereader as shapereader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature.nightshade import Nightshade
import pyproj
from shapely.geometry import Polygon, mapping
from shapely.geometry import Point, Polygon
from shapely import wkt

ModuleNotFoundError: ignored

In [ ]:
ruta = 'https://foscalinternacional.com/datasets/eventos_dengue_2020.csv'
df = pd.read_csv(ruta)

# Creación de la proyección
P = pyproj.Proj(proj='utm', zone=18, ellps='WGS84', preserve_units=True)

# Se convertirán las coordenadas geográficas a UTM
df['x'], df['y'] = P(df['longitud'], df['latitud'])

# Obtain the remote shapefile files
url = 'https://foscal.net/cartografia/'

file_list= ["68276_urbano.cpg","68276_urbano.dbf","68276_urbano.prj","68276_urbano.qmd","68276_urbano.shp","68276_urbano.shx"]

for file in file_list:
  urllib.request.urlretrieve(url+file, file)

In [ ]:
subplot_kw = dict(projection=ccrs.PlateCarree())

fig, ax = plt.subplots(figsize=(16, 16), subplot_kw=subplot_kw)

# Mapa de floridablanca
lower_lon = -73.135
upper_lon = -73.070
lower_lat = 7.051
upper_lat = 7.110

ax.set_extent([lower_lon, upper_lon, lower_lat, upper_lat])

reader = shapereader.Reader('68276_urbano.shp')

for geometria in reader.geometries():
    ax.add_geometries([geometria], ccrs.PlateCarree(), facecolor='white', edgecolor='gray')

ax.scatter(df['longitud'], df['latitud'], zorder=10, c='b', s=10)

ax.add_line
plt.show()

In [ ]:
for geometria in reader.geometries():
  geo = geometria

geod = pyproj.Geod(ellps="WGS84")
area = abs(geod.geometry_area_perimeter(geo)[0])

In [ ]:
reader = shapereader.Reader('68276_urbano.shp')
points_to_use = df.shape[0]

for geometry in reader.geometries():
  polygon = geometry

def random_points_in_bound(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return x, y

In [ ]:
import geopandas as gpd

gdf_poly = gpd.GeoDataFrame(index=["florida"], geometry=[polygon])

In [ ]:
import pandas as pd

def generate_random_points():
  x,y = random_points_in_bound(polygon, 10_000)
  dfGen = pd.DataFrame()
  dfGen['points'] = list(zip(x,y))
  dfGen['points'] = dfGen['points'].apply(Point)
  gdf_points = gpd.GeoDataFrame(dfGen, geometry='points')

  Sjoin = gpd.tools.sjoin(gdf_points, gdf_poly, predicate="within", how='left')

  pnts_in_poly = gdf_points[Sjoin.index_right=='florida']
  pnts_in_poly.index = range(0,len(pnts_in_poly))
  pnts_in_poly = dfGen[dfGen.index < points_to_use]
  pnts_in_poly["x"] = [point.x for point in pnts_in_poly["points"].to_list()]
  pnts_in_poly["y"] = [point.y for point in pnts_in_poly["points"].to_list()]

  return pnts_in_poly

In [ ]:
generate_random_points()

In [ ]:
def distance(x1, y1, x2, y2):
  return sqrt((x2-x1)**2 + (y2-y1)**2)

In [ ]:
def puntosCercanos (pointList, h):
  counter = 0
  pointOneIndex = 0
  for pointOne in pointList:
    if(pointOneIndex +1 < len(pointList)):
      for pointTwo in pointList[pointOneIndex + 1:len(pointList)]:
        if (distance(pointOne[0],pointOne[1],pointTwo[0],pointTwo[1]) <= h):
          counter += 1
    pointOneIndex += 1
  return counter
def valorK(puntos, area, h):
  return (area/(len(puntos)**2)) * puntosCercanos(puntos,h)

In [ ]:
def kCompleto(originalPointList, ranList, h):
  #print(ranList)
  result = []
  originalK = valorK(originalPointList, area, h)
  result.append(originalK)

  totalPoints = len(originalPointList)
  for randSet in ranList:
    result.append(valorK(randSet, area, h))
  result.sort()
  return [h, originalK, np.percentile(result, 2.5), np.percentile(result, 97.5)]

In [ ]:
import time

def iterateK(originalPointList):
  result = []
  randList = []
  totalPoints = len(originalPointList)
  for i in range(999):
    print("\rDatos Aleatorios = " + str(i + 1), end='', flush=True)
    
    #ye = generadorPuntos(totalPoints,poly)
    randList.append(generate_random_points())
  print()
  for i in range(5,100,5):
    start_time = time.time()
    print(str(i) + " - " + str(time.time()-start_time))
    result.append(kCompleto(originalPointList, randList, i))
  return result